# Scrape

In [1]:
from splinter import Browser
from bs4 import BeautifulSoup 
import pandas as pd
import time
import requests

In [2]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

### Mars News

In [3]:
# Open browser to NASA Mars News Site
browser.visit('https://mars.nasa.gov/news/')

In [4]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

article = soup.find("div", class_="list_text")
news_p = article.find("div", class_="article_teaser_body").text
news_title = article.find("div", class_="content_title").text
news_date = article.find("div", class_="list_date").text
print(news_date)
print(news_title)
print(news_p)

March  5, 2020
Virginia Middle School Student Earns Honor of Naming NASA's Next Mars Rover
NASA chose a seventh-grader from Virginia as winner of the agency's "Name the Rover" essay contest. Alexander Mather's entry for "Perseverance" was voted tops among 28,000 entries. 


### Mars Featured Imaged

In [5]:
# Open browser to JPL Featured Image
browser.visit('https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars')

In [6]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
image = soup.find("img", class_="thumb")["src"]
img_url = "https://jpl.nasa.gov"+image
featured_image_url = img_url  
f'featured_image_url = {featured_image_url}'

print(featured_image_url)

https://jpl.nasa.gov/spaceimages/images/wallpaper/PIA23745-640x350.jpg


### Mars Weather

In [7]:
# Open browser to Mars Weather Twitter Account
browser.visit('https://twitter.com/marswxreport?lang=en')

In [8]:
url = "https://twitter.com/marswxreport?lang=en"
response = requests.get(url)
bsoup = BeautifulSoup(response.text, "html.parser")

mars_weather = bsoup.find("div", class_="js-tweet-text-container").text.strip()

print(mars_weather)

InSight sol 457 (2020-03-10) low -95.7ºC (-140.3ºF) high -9.1ºC (15.6ºF)
winds from the SSE at 6.5 m/s (14.5 mph) gusting to 21.0 m/s (46.9 mph)
pressure at 6.30 hPapic.twitter.com/2h0LKoSQrJ


### Mars Facts

In [ ]:
# Use Pandas to scrape data
url3 = "http://space-facts.com/mars/"
browser.visit(url3)

In [ ]:
# Convert table to html
import pandas as pd 
grab=pd.read_html(url3)
mars_data=pd.DataFrame(grab[0])
mars_data.columns=['Mars','Data']
mars_table=mars_data.set_index("Mars")
marsdata = mars_table.to_html(classes='marsdata')
marsdata=marsdata.replace('\n', ' ')
marsdata

### Mars Hemispheres

In [ ]:
# Open browser to USGS Astrogeology site
browser.visit('https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars')

#### Search for hemisphere titles

In [ ]:
html = browser.html
soup = bs(html, 'html.parser')

hemi_names = []

# Search for the names of all four hemispheres
results = soup.find_all('div', class_="collapsible results")
hemispheres = results[0].find_all('h3')

# Get text and store in list
for name in hemispheres:
    hemi_names.append(name.text)

hemi_names

#### Click through thumbnails to get full-sized hemisphere images

In [ ]:
# Search for thumbnail links
thumbnail_results = results[0].find_all('a')
thumbnail_links = []

for thumbnail in thumbnail_results:
    
    # If the thumbnail element has an image...
    if (thumbnail.img):
        
        # then grab the attached link
        thumbnail_url = 'https://astrogeology.usgs.gov/' + thumbnail['href']
        
        # Append list with links
        thumbnail_links.append(thumbnail_url)

thumbnail_links

#### Extract image source of full-sized image

In [ ]:
full_imgs = []

for url in thumbnail_links:
    
    # Click through each thumbanil link
    browser.visit(url)
    
    html = browser.html
    soup = bs(html, 'html.parser')
    
    # Scrape each page for the relative image path
    results = soup.find_all('img', class_='wide-image')
    relative_img_path = results[0]['src']
    
    # Combine the reltaive image path to get the full url
    img_link = 'https://astrogeology.usgs.gov/' + relative_img_path
    
    # Add full image links to a list
    full_imgs.append(img_link)

full_imgs

#### Store as a list of dictionaries

In [ ]:
# Zip together the list of hemisphere names and hemisphere image links
mars_hemi_zip = zip(hemi_names, full_imgs)

hemisphere_image_urls = []

# Iterate through the zipped object
for title, img in mars_hemi_zip:
    
    mars_hemi_dict = {}
    
    # Add hemisphere title to dictionary
    mars_hemi_dict['title'] = title
    
    # Add image url to dictionary
    mars_hemi_dict['img_url'] = img
    
    # Append the list with dictionaries
    hemisphere_image_urls.append(mars_hemi_dict)

hemisphere_image_urls